In [1]:
from selenium import webdriver
import json
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
def web_scrape(url):
    options = Options()
    options.add_argument("--enable-performance-logging")
    options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    log = driver.get_log("performance")
    driver.quit()
    
    return log

In [3]:
url = "https://www.oddsportal.com/football/belgium/jupiler-pro-league/"
scraped = web_scrape(url)

In [4]:
with open("oddsportal_network_dump.txt", 'w') as f:
    for i in scraped:
        test = i['message']
        json_obj = json.loads(test)
        json_formatted = json.dumps(json_obj, indent=2)
        f.write(json_formatted)
    f.close()